Youtube Trend Analysis
Which months, time of the day, day of the week are most popular for uploads?
Who are the top channels?
What are the most common words used in titles? What is the relationship between these words and their views?
What are the most popular tags? Which are the most liked / disliked tags?
What kinds of videos get the most comments?
How does the sentiment of the description correlate with views, comments, likes/dislikes?
Preprocessing:
Create date columns: hour, day, month
Parse keywords from titles, lowercase
Parse keywords from tags
Link category ids to categories
Analysis:
Find trends between comments, likes, dislikes and channel / date / tags /// esto es de christine, cambiar! y VER SI AÑADIR GITIGNORE


https://www.kaggle.com/datasets/yusufdelikkaya/online-sales-dataset? SOURCE

### Load dataset, preprocessing

In [1]:
## Import libraries: 

import pandas as pd
import numpy as np
from src import functions as func
import re
# Viz
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# ------------------------------------------------------------------------------
#from scipy.stats import shapiro, kstest, poisson, chisquare, ttest_ind, levene, bartlett, sem, ppf
import scipy.stats as stats
from scipy.stats import shapiro, levene
from scipy.stats import ttest_ind
from scipy.stats import mannwhitneyu
from scipy.stats import chi2_contingency

# Configuration 
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # to viz all columns in the DF

# Manage warnings: 
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")

In [2]:
online_sales = pd.read_csv('data/online_sales_dataset.csv')
online_sales.head(2)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Discount,PaymentMethod,ShippingCost,Category,SalesChannel,ReturnStatus,ShipmentProvider,WarehouseLocation,OrderPriority
0,221958,SKU_1964,White Mug,38,2020-01-01 00:00,1.71,37039.0,Australia,0.47,Bank Transfer,10.79,Apparel,In-store,Not Returned,UPS,London,Medium
1,771155,SKU_1241,White Mug,18,2020-01-01 01:00,41.25,19144.0,Spain,0.19,paypall,9.51,Electronics,Online,Not Returned,UPS,Rome,Medium


In [3]:
## EDA: 

func.exploration(online_sales)

The DataFrame has 49782 rows and 17 columns.
It has 0 duplicate records, which accounts for 0.0% of the data. 

There are 3 columns with missing data, and they are: 
['CustomerID', 'ShippingCost', 'WarehouseLocation']

And with no missing values, there are 14 columns, and they are: 
['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'UnitPrice', 'Country', 'Discount', 'PaymentMethod', 'Category', 'SalesChannel', 'ReturnStatus', 'ShipmentProvider', 'OrderPriority']


Below is a breakdown of missing data, data types, and the number of values:


,% missing,% non-missing,data_type,num_unique_values
InvoiceNo,0.0%,100.0%,int64,48494
StockCode,0.0%,100.0%,object,1000
Description,0.0%,100.0%,object,11
Quantity,0.0%,100.0%,int64,99
InvoiceDate,0.0%,100.0%,object,49782


Main statistics for categorical columns:


,count,unique,top,freq
StockCode,49782,1000,SKU_1761,76
Description,49782,11,Wall Clock,4617
InvoiceDate,49782,49782,2025-09-05 05:00,1
Country,49782,12,France,4230
PaymentMethod,49782,3,Bank Transfer,16747
Category,49782,5,Furniture,10084
SalesChannel,49782,2,Online,25051
ReturnStatus,49782,2,Not Returned,44888
ShipmentProvider,49782,4,FedEx,12501
WarehouseLocation,46297,5,Amsterdam,9458


Main statistics for numerical columns:


,count,mean,std,min,25%,50%,75%,max
InvoiceNo,49782.0,550681.239946,260703.009944,100005.00,324543.0000,552244.00,776364.00,999997.000000
Quantity,49782.0,22.372343,17.917774,-50.00,11.0000,23.00,37.00,49.000000
UnitPrice,49782.0,47.537862,33.479510,-99.98,23.5925,48.92,74.61,100.000000
CustomerID,44804.0,55032.871775,25913.660157,10001.00,32750.7500,55165.00,77306.25,99998.000000
Discount,49782.0,0.275748,0.230077,0.00,0.1300,0.26,0.38,1.999764
ShippingCost,47293.0,17.494529,7.220557,5.00,11.2200,17.50,23.72,30.000000


,% missing,% non-missing,data_type,num_unique_values
InvoiceNo,0.0%,100.0%,int64,48494
StockCode,0.0%,100.0%,object,1000
Description,0.0%,100.0%,object,11
Quantity,0.0%,100.0%,int64,99
InvoiceDate,0.0%,100.0%,object,49782
UnitPrice,0.0%,100.0%,float64,11200
CustomerID,10.0%,90.0%,float64,35389
Country,0.0%,100.0%,object,12
Discount,0.0%,100.0%,float64,1047
PaymentMethod,0.0%,100.0%,object,3


In [4]:
online_sales.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country', 'Discount', 'PaymentMethod',
       'ShippingCost', 'Category', 'SalesChannel', 'ReturnStatus',
       'ShipmentProvider', 'WarehouseLocation', 'OrderPriority'],
      dtype='object')

In [5]:
func.separate_columns(online_sales)

,Invoice No,Stock Code,Description,Quantity,Invoice Date,Unit Price,Customer ID,Country,Discount,Payment Method,Shipping Cost,Category,Sales Channel,Return Status,Shipment Provider,Warehouse Location,Order Priority
0,221958,SKU_1964,White Mug,38,2020-01-01 00:00,1.71,37039.0,Australia,0.470000,Bank Transfer,10.79,Apparel,In-store,Not Returned,UPS,London,Medium
1,771155,SKU_1241,White Mug,18,2020-01-01 01:00,41.25,19144.0,Spain,0.190000,paypall,9.51,Electronics,Online,Not Returned,UPS,Rome,Medium
2,231932,SKU_1501,Headphones,49,2020-01-01 02:00,29.11,50472.0,Germany,0.350000,Bank Transfer,23.03,Electronics,Online,Returned,UPS,Berlin,High
3,465838,SKU_1760,Desk Lamp,14,2020-01-01 03:00,76.68,96586.0,Netherlands,0.140000,paypall,11.08,Accessories,Online,Not Returned,Royal Mail,Rome,Low
4,359178,SKU_1386,USB Cable,-30,2020-01-01 04:00,-68.11,NaN,United Kingdom,1.501433,Bank Transfer,NaN,Electronics,In-store,Not Returned,FedEx,NaN,Medium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49777,354083,SKU_1562,Blue Pen,25,2025-09-05 01:00,70.92,51445.0,Spain,0.200000,Credit Card,8.96,Electronics,Online,Returned,UPS,Berlin,Medium
49778,296698,SKU_1930,USB Cable,7,2025-09-05 02:00,51.74,28879.0,United States,0.230000,Bank Transfer,23.55,Electronics,Online,Not Returned,FedEx,Amsterdam,Low
49779,177622,SKU_1766,Office Chair,43,2025-09-05 03:00,85.25,21825.0,Portugal,0.200000,Bank Transfer,16.26,Furniture,In-store,Not Returned,FedEx,London,High
49780,701213,SKU_1602,Notebook,48,2025-09-05 04:00,39.64,43199.0,United Kingdom,0.310000,paypall,28.56,Apparel,Online,Not Returned,Royal Mail,London,Medium


In [6]:
## Checking result: 

online_sales.columns

Index(['Invoice No', 'Stock Code', 'Description', 'Quantity', 'Invoice Date',
       'Unit Price', 'Customer ID', 'Country', 'Discount', 'Payment Method',
       'Shipping Cost', 'Category', 'Sales Channel', 'Return Status',
       'Shipment Provider', 'Warehouse Location', 'Order Priority'],
      dtype='object')

In [7]:
## Split invoice date column in 2 (date & time), drop original column and time column, and convert date to datetime: 

func.transform_datetime_columns(online_sales,'Invoice Date')

,Invoice No,Stock Code,Description,Quantity,Unit Price,Customer ID,Country,Discount,Payment Method,Shipping Cost,Category,Sales Channel,Return Status,Shipment Provider,Warehouse Location,Order Priority,Order Date
0,221958,SKU_1964,White Mug,38,1.71,37039.0,Australia,0.470000,Bank Transfer,10.79,Apparel,In-store,Not Returned,UPS,London,Medium,2020-01-01
1,771155,SKU_1241,White Mug,18,41.25,19144.0,Spain,0.190000,paypall,9.51,Electronics,Online,Not Returned,UPS,Rome,Medium,2020-01-01
2,231932,SKU_1501,Headphones,49,29.11,50472.0,Germany,0.350000,Bank Transfer,23.03,Electronics,Online,Returned,UPS,Berlin,High,2020-01-01
3,465838,SKU_1760,Desk Lamp,14,76.68,96586.0,Netherlands,0.140000,paypall,11.08,Accessories,Online,Not Returned,Royal Mail,Rome,Low,2020-01-01
4,359178,SKU_1386,USB Cable,-30,-68.11,NaN,United Kingdom,1.501433,Bank Transfer,NaN,Electronics,In-store,Not Returned,FedEx,NaN,Medium,2020-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49777,354083,SKU_1562,Blue Pen,25,70.92,51445.0,Spain,0.200000,Credit Card,8.96,Electronics,Online,Returned,UPS,Berlin,Medium,2025-09-05
49778,296698,SKU_1930,USB Cable,7,51.74,28879.0,United States,0.230000,Bank Transfer,23.55,Electronics,Online,Not Returned,FedEx,Amsterdam,Low,2025-09-05
49779,177622,SKU_1766,Office Chair,43,85.25,21825.0,Portugal,0.200000,Bank Transfer,16.26,Furniture,In-store,Not Returned,FedEx,London,High,2025-09-05
49780,701213,SKU_1602,Notebook,48,39.64,43199.0,United Kingdom,0.310000,paypall,28.56,Apparel,Online,Not Returned,Royal Mail,London,Medium,2025-09-05


In [8]:
#'Customer ID' column 

# Replace NaN (10%) values with a specific integer value (0) and convert the column to int.

online_sales['Customer ID'] = online_sales['Customer ID'].fillna(0).astype(int)


In [9]:
online_sales['Customer ID'].value_counts()

Customer ID
0        4978
53125       6
20524       5
81631       5
91749       5
         ... 
36515       1
63980       1
85250       1
12024       1
39718       1
Name: count, Length: 35390, dtype: int64

In [10]:
df_test = online_sales[online_sales['Return Status'] == 'Returned']

In [11]:
df_test = online_sales[online_sales['Unit Price'] < 0]

In [12]:
df_test

,Invoice No,Stock Code,Description,Quantity,Unit Price,Customer ID,Country,Discount,Payment Method,Shipping Cost,Category,Sales Channel,Return Status,Shipment Provider,Warehouse Location,Order Priority,Order Date
4,359178,SKU_1386,USB Cable,-30,-68.11,0,United Kingdom,1.501433,Bank Transfer,NaN,Electronics,In-store,Not Returned,FedEx,NaN,Medium,2020-01-01
147,747972,SKU_1514,Desk Lamp,-26,-72.33,0,Netherlands,1.972087,paypall,NaN,Electronics,In-store,Not Returned,Royal Mail,NaN,Low,2020-01-07
167,318969,SKU_1115,Desk Lamp,-19,-3.61,0,Italy,1.052254,Credit Card,NaN,Apparel,In-store,Not Returned,UPS,NaN,High,2020-01-07
199,867595,SKU_1343,White Mug,-2,-96.08,0,Portugal,1.702088,Bank Transfer,NaN,Accessories,Online,Not Returned,FedEx,NaN,Low,2020-01-09
262,937291,SKU_1936,Wall Clock,-16,-73.60,0,Germany,1.535440,Credit Card,NaN,Electronics,Online,Not Returned,DHL,NaN,Medium,2020-01-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49691,553510,SKU_1943,T-shirt,-4,-74.57,0,France,1.306613,Credit Card,NaN,Stationery,Online,Not Returned,Royal Mail,NaN,Low,2025-09-01
49695,855821,SKU_1500,Wireless Mouse,-11,-18.88,0,Portugal,1.569082,Bank Transfer,NaN,Furniture,In-store,Not Returned,FedEx,NaN,Low,2025-09-01
49718,745888,SKU_1224,T-shirt,-2,-84.25,0,United States,1.420665,Bank Transfer,NaN,Accessories,In-store,Not Returned,Royal Mail,NaN,Medium,2025-09-02
49730,487771,SKU_1083,Wall Clock,-7,-57.31,0,United Kingdom,1.139869,paypall,NaN,Accessories,Online,Not Returned,DHL,NaN,Low,2025-09-03


In [13]:
### Analyzing negative values in Unit Price & Quantity columns

In [14]:
# Neg values on Unit Price column: 

unit_price_neg = online_sales[online_sales['Unit Price'] < 0]

In [15]:
unit_price_neg

,Invoice No,Stock Code,Description,Quantity,Unit Price,Customer ID,Country,Discount,Payment Method,Shipping Cost,Category,Sales Channel,Return Status,Shipment Provider,Warehouse Location,Order Priority,Order Date
4,359178,SKU_1386,USB Cable,-30,-68.11,0,United Kingdom,1.501433,Bank Transfer,NaN,Electronics,In-store,Not Returned,FedEx,NaN,Medium,2020-01-01
147,747972,SKU_1514,Desk Lamp,-26,-72.33,0,Netherlands,1.972087,paypall,NaN,Electronics,In-store,Not Returned,Royal Mail,NaN,Low,2020-01-07
167,318969,SKU_1115,Desk Lamp,-19,-3.61,0,Italy,1.052254,Credit Card,NaN,Apparel,In-store,Not Returned,UPS,NaN,High,2020-01-07
199,867595,SKU_1343,White Mug,-2,-96.08,0,Portugal,1.702088,Bank Transfer,NaN,Accessories,Online,Not Returned,FedEx,NaN,Low,2020-01-09
262,937291,SKU_1936,Wall Clock,-16,-73.60,0,Germany,1.535440,Credit Card,NaN,Electronics,Online,Not Returned,DHL,NaN,Medium,2020-01-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49691,553510,SKU_1943,T-shirt,-4,-74.57,0,France,1.306613,Credit Card,NaN,Stationery,Online,Not Returned,Royal Mail,NaN,Low,2025-09-01
49695,855821,SKU_1500,Wireless Mouse,-11,-18.88,0,Portugal,1.569082,Bank Transfer,NaN,Furniture,In-store,Not Returned,FedEx,NaN,Low,2025-09-01
49718,745888,SKU_1224,T-shirt,-2,-84.25,0,United States,1.420665,Bank Transfer,NaN,Accessories,In-store,Not Returned,Royal Mail,NaN,Medium,2025-09-02
49730,487771,SKU_1083,Wall Clock,-7,-57.31,0,United Kingdom,1.139869,paypall,NaN,Accessories,Online,Not Returned,DHL,NaN,Low,2025-09-03


In [16]:
# Neg values on Quantity column: 

quantity_neg = online_sales[online_sales['Quantity'] < 0]

In [17]:
quantity_neg

,Invoice No,Stock Code,Description,Quantity,Unit Price,Customer ID,Country,Discount,Payment Method,Shipping Cost,Category,Sales Channel,Return Status,Shipment Provider,Warehouse Location,Order Priority,Order Date
4,359178,SKU_1386,USB Cable,-30,-68.11,0,United Kingdom,1.501433,Bank Transfer,NaN,Electronics,In-store,Not Returned,FedEx,NaN,Medium,2020-01-01
144,451279,SKU_1617,Wall Clock,-2,34.06,0,Spain,0.340000,Credit Card,NaN,Furniture,Online,Not Returned,DHL,NaN,Medium,2020-01-07
147,747972,SKU_1514,Desk Lamp,-26,-72.33,0,Netherlands,1.972087,paypall,NaN,Electronics,In-store,Not Returned,Royal Mail,NaN,Low,2020-01-07
167,318969,SKU_1115,Desk Lamp,-19,-3.61,0,Italy,1.052254,Credit Card,NaN,Apparel,In-store,Not Returned,UPS,NaN,High,2020-01-07
176,285340,SKU_1710,Blue Pen,-3,85.95,0,Netherlands,0.260000,Credit Card,NaN,Accessories,In-store,Not Returned,UPS,NaN,Low,2020-01-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49730,487771,SKU_1083,Wall Clock,-7,-57.31,0,United Kingdom,1.139869,paypall,NaN,Accessories,Online,Not Returned,DHL,NaN,Low,2025-09-03
49739,106493,SKU_1736,Desk Lamp,-40,-51.34,0,Netherlands,0.230000,Bank Transfer,NaN,Accessories,In-store,Not Returned,FedEx,NaN,Medium,2025-09-03
49749,540459,SKU_1405,White Mug,-36,12.21,0,Portugal,0.290000,paypall,NaN,Stationery,Online,Not Returned,UPS,NaN,High,2025-09-03
49760,726763,SKU_1273,White Mug,-34,10.34,0,France,0.350000,Credit Card,NaN,Furniture,Online,Not Returned,DHL,NaN,High,2025-09-04


In [18]:
## Checking if it could be related to 'Return Status' = Returned

In [19]:
df_test1 = online_sales[(online_sales['Quantity'] < 0) & (online_sales['Return Status'] == 'Returned')]


In [20]:
df_test2 = online_sales[(online_sales['Unit Price'] < 0) & (online_sales['Return Status'] == 'Returned')]


In [21]:
df_test1.head()

,Invoice No,Stock Code,Description,Quantity,Unit Price,Customer ID,Country,Discount,Payment Method,Shipping Cost,Category,Sales Channel,Return Status,Shipment Provider,Warehouse Location,Order Priority,Order Date
399,760960,SKU_1535,Notebook,-15,27.30,0,Sweden,0.410000,paypall,NaN,Accessories,In-store,Returned,FedEx,NaN,Low,2020-01-17
454,258823,SKU_1229,White Mug,-47,98.15,0,Germany,0.140000,Credit Card,NaN,Apparel,In-store,Returned,DHL,NaN,High,2020-01-19
465,544209,SKU_1717,White Mug,-43,-1.47,0,France,1.330014,Bank Transfer,NaN,Electronics,Online,Returned,UPS,NaN,High,2020-01-20
469,319963,SKU_1411,Notebook,-27,51.17,0,United Kingdom,0.000000,paypall,NaN,Electronics,In-store,Returned,Royal Mail,NaN,Medium,2020-01-20
501,489957,SKU_1542,USB Cable,-11,-84.13,0,United States,0.190000,Bank Transfer,NaN,Accessories,In-store,Returned,DHL,NaN,Medium,2020-01-21


In [22]:
df_test2.head()

,Invoice No,Stock Code,Description,Quantity,Unit Price,Customer ID,Country,Discount,Payment Method,Shipping Cost,Category,Sales Channel,Return Status,Shipment Provider,Warehouse Location,Order Priority,Order Date
465,544209,SKU_1717,White Mug,-43,-1.47,0,France,1.330014,Bank Transfer,NaN,Electronics,Online,Returned,UPS,NaN,High,2020-01-20
501,489957,SKU_1542,USB Cable,-11,-84.13,0,United States,0.190000,Bank Transfer,NaN,Accessories,In-store,Returned,DHL,NaN,Medium,2020-01-21
708,825256,SKU_1862,Backpack,-35,-86.53,0,France,1.348446,Bank Transfer,NaN,Electronics,Online,Returned,FedEx,NaN,Medium,2020-01-30
1114,101062,SKU_1594,White Mug,-32,-29.62,0,Italy,0.260000,paypall,NaN,Electronics,Online,Returned,FedEx,NaN,Low,2020-02-16
1388,938761,SKU_1434,Desk Lamp,-33,-22.69,0,Italy,1.561084,Credit Card,NaN,Stationery,Online,Returned,UPS,NaN,High,2020-02-27


In [23]:
## Not related. 

In [24]:
## Checking if it could be related to 'Warehouse Location' = NaN

In [25]:
df_test3 = online_sales[(online_sales['Unit Price'] < 0) & (online_sales['Warehouse Location'].isna())]
df_test4 = online_sales[(online_sales['Quantity'] < 0) & (online_sales['Warehouse Location'].isna())]

In [26]:
df_test3

,Invoice No,Stock Code,Description,Quantity,Unit Price,Customer ID,Country,Discount,Payment Method,Shipping Cost,Category,Sales Channel,Return Status,Shipment Provider,Warehouse Location,Order Priority,Order Date
4,359178,SKU_1386,USB Cable,-30,-68.11,0,United Kingdom,1.501433,Bank Transfer,NaN,Electronics,In-store,Not Returned,FedEx,NaN,Medium,2020-01-01
147,747972,SKU_1514,Desk Lamp,-26,-72.33,0,Netherlands,1.972087,paypall,NaN,Electronics,In-store,Not Returned,Royal Mail,NaN,Low,2020-01-07
167,318969,SKU_1115,Desk Lamp,-19,-3.61,0,Italy,1.052254,Credit Card,NaN,Apparel,In-store,Not Returned,UPS,NaN,High,2020-01-07
199,867595,SKU_1343,White Mug,-2,-96.08,0,Portugal,1.702088,Bank Transfer,NaN,Accessories,Online,Not Returned,FedEx,NaN,Low,2020-01-09
262,937291,SKU_1936,Wall Clock,-16,-73.60,0,Germany,1.535440,Credit Card,NaN,Electronics,Online,Not Returned,DHL,NaN,Medium,2020-01-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49691,553510,SKU_1943,T-shirt,-4,-74.57,0,France,1.306613,Credit Card,NaN,Stationery,Online,Not Returned,Royal Mail,NaN,Low,2025-09-01
49695,855821,SKU_1500,Wireless Mouse,-11,-18.88,0,Portugal,1.569082,Bank Transfer,NaN,Furniture,In-store,Not Returned,FedEx,NaN,Low,2025-09-01
49718,745888,SKU_1224,T-shirt,-2,-84.25,0,United States,1.420665,Bank Transfer,NaN,Accessories,In-store,Not Returned,Royal Mail,NaN,Medium,2025-09-02
49730,487771,SKU_1083,Wall Clock,-7,-57.31,0,United Kingdom,1.139869,paypall,NaN,Accessories,Online,Not Returned,DHL,NaN,Low,2025-09-03


In [27]:
df_test4

,Invoice No,Stock Code,Description,Quantity,Unit Price,Customer ID,Country,Discount,Payment Method,Shipping Cost,Category,Sales Channel,Return Status,Shipment Provider,Warehouse Location,Order Priority,Order Date
4,359178,SKU_1386,USB Cable,-30,-68.11,0,United Kingdom,1.501433,Bank Transfer,NaN,Electronics,In-store,Not Returned,FedEx,NaN,Medium,2020-01-01
144,451279,SKU_1617,Wall Clock,-2,34.06,0,Spain,0.340000,Credit Card,NaN,Furniture,Online,Not Returned,DHL,NaN,Medium,2020-01-07
147,747972,SKU_1514,Desk Lamp,-26,-72.33,0,Netherlands,1.972087,paypall,NaN,Electronics,In-store,Not Returned,Royal Mail,NaN,Low,2020-01-07
167,318969,SKU_1115,Desk Lamp,-19,-3.61,0,Italy,1.052254,Credit Card,NaN,Apparel,In-store,Not Returned,UPS,NaN,High,2020-01-07
176,285340,SKU_1710,Blue Pen,-3,85.95,0,Netherlands,0.260000,Credit Card,NaN,Accessories,In-store,Not Returned,UPS,NaN,Low,2020-01-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49730,487771,SKU_1083,Wall Clock,-7,-57.31,0,United Kingdom,1.139869,paypall,NaN,Accessories,Online,Not Returned,DHL,NaN,Low,2025-09-03
49739,106493,SKU_1736,Desk Lamp,-40,-51.34,0,Netherlands,0.230000,Bank Transfer,NaN,Accessories,In-store,Not Returned,FedEx,NaN,Medium,2025-09-03
49749,540459,SKU_1405,White Mug,-36,12.21,0,Portugal,0.290000,paypall,NaN,Stationery,Online,Not Returned,UPS,NaN,High,2025-09-03
49760,726763,SKU_1273,White Mug,-34,10.34,0,France,0.350000,Credit Card,NaN,Furniture,Online,Not Returned,DHL,NaN,High,2025-09-04


In [28]:
warehouse_neg = online_sales[online_sales['Warehouse Location'].isna()]

In [29]:
warehouse_neg

,Invoice No,Stock Code,Description,Quantity,Unit Price,Customer ID,Country,Discount,Payment Method,Shipping Cost,Category,Sales Channel,Return Status,Shipment Provider,Warehouse Location,Order Priority,Order Date
4,359178,SKU_1386,USB Cable,-30,-68.11,0,United Kingdom,1.501433,Bank Transfer,NaN,Electronics,In-store,Not Returned,FedEx,NaN,Medium,2020-01-01
62,305041,SKU_1731,Desk Lamp,45,54.79,0,United Kingdom,0.230000,Credit Card,8.97,Accessories,In-store,Not Returned,DHL,NaN,Medium,2020-01-03
103,124300,SKU_1951,Headphones,8,46.19,0,Norway,0.100000,Bank Transfer,12.81,Stationery,In-store,Not Returned,UPS,NaN,High,2020-01-05
144,451279,SKU_1617,Wall Clock,-2,34.06,0,Spain,0.340000,Credit Card,NaN,Furniture,Online,Not Returned,DHL,NaN,Medium,2020-01-07
147,747972,SKU_1514,Desk Lamp,-26,-72.33,0,Netherlands,1.972087,paypall,NaN,Electronics,In-store,Not Returned,Royal Mail,NaN,Low,2020-01-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49739,106493,SKU_1736,Desk Lamp,-40,-51.34,0,Netherlands,0.230000,Bank Transfer,NaN,Accessories,In-store,Not Returned,FedEx,NaN,Medium,2025-09-03
49749,540459,SKU_1405,White Mug,-36,12.21,0,Portugal,0.290000,paypall,NaN,Stationery,Online,Not Returned,UPS,NaN,High,2025-09-03
49760,726763,SKU_1273,White Mug,-34,10.34,0,France,0.350000,Credit Card,NaN,Furniture,Online,Not Returned,DHL,NaN,High,2025-09-04
49761,829071,SKU_1000,Wall Clock,1,27.81,0,Sweden,0.390000,Bank Transfer,7.40,Apparel,In-store,Not Returned,DHL,NaN,Low,2025-09-04


In [30]:
## Not related (996 rows have a null value for Warehouse Location but there is a positive value on Unit price or Quantity)

In [31]:
## Checking if it's related to null values on 'Shipping Cost' column 

In [32]:
shipping_cost_neg = online_sales[online_sales['Shipping Cost'].isna()]

In [33]:
shipping_cost_neg

,Invoice No,Stock Code,Description,Quantity,Unit Price,Customer ID,Country,Discount,Payment Method,Shipping Cost,Category,Sales Channel,Return Status,Shipment Provider,Warehouse Location,Order Priority,Order Date
4,359178,SKU_1386,USB Cable,-30,-68.11,0,United Kingdom,1.501433,Bank Transfer,NaN,Electronics,In-store,Not Returned,FedEx,NaN,Medium,2020-01-01
144,451279,SKU_1617,Wall Clock,-2,34.06,0,Spain,0.340000,Credit Card,NaN,Furniture,Online,Not Returned,DHL,NaN,Medium,2020-01-07
147,747972,SKU_1514,Desk Lamp,-26,-72.33,0,Netherlands,1.972087,paypall,NaN,Electronics,In-store,Not Returned,Royal Mail,NaN,Low,2020-01-07
167,318969,SKU_1115,Desk Lamp,-19,-3.61,0,Italy,1.052254,Credit Card,NaN,Apparel,In-store,Not Returned,UPS,NaN,High,2020-01-07
176,285340,SKU_1710,Blue Pen,-3,85.95,0,Netherlands,0.260000,Credit Card,NaN,Accessories,In-store,Not Returned,UPS,NaN,Low,2020-01-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49730,487771,SKU_1083,Wall Clock,-7,-57.31,0,United Kingdom,1.139869,paypall,NaN,Accessories,Online,Not Returned,DHL,NaN,Low,2025-09-03
49739,106493,SKU_1736,Desk Lamp,-40,-51.34,0,Netherlands,0.230000,Bank Transfer,NaN,Accessories,In-store,Not Returned,FedEx,NaN,Medium,2025-09-03
49749,540459,SKU_1405,White Mug,-36,12.21,0,Portugal,0.290000,paypall,NaN,Stationery,Online,Not Returned,UPS,NaN,High,2025-09-03
49760,726763,SKU_1273,White Mug,-34,10.34,0,France,0.350000,Credit Card,NaN,Furniture,Online,Not Returned,DHL,NaN,High,2025-09-04


In [34]:
## Number of rows match

## Checking if rows are similar across the 3 DF's to confirm the root cause. 

In [35]:
## Checking how many rows are when the Unit Price is negative and the Shipping Cost is NaN (1493 rows)
filtered_df1 = online_sales[(online_sales['Unit Price'] < 0) & (online_sales['Shipping Cost'].isna())]



In [36]:
filtered_df1

,Invoice No,Stock Code,Description,Quantity,Unit Price,Customer ID,Country,Discount,Payment Method,Shipping Cost,Category,Sales Channel,Return Status,Shipment Provider,Warehouse Location,Order Priority,Order Date
4,359178,SKU_1386,USB Cable,-30,-68.11,0,United Kingdom,1.501433,Bank Transfer,NaN,Electronics,In-store,Not Returned,FedEx,NaN,Medium,2020-01-01
147,747972,SKU_1514,Desk Lamp,-26,-72.33,0,Netherlands,1.972087,paypall,NaN,Electronics,In-store,Not Returned,Royal Mail,NaN,Low,2020-01-07
167,318969,SKU_1115,Desk Lamp,-19,-3.61,0,Italy,1.052254,Credit Card,NaN,Apparel,In-store,Not Returned,UPS,NaN,High,2020-01-07
199,867595,SKU_1343,White Mug,-2,-96.08,0,Portugal,1.702088,Bank Transfer,NaN,Accessories,Online,Not Returned,FedEx,NaN,Low,2020-01-09
262,937291,SKU_1936,Wall Clock,-16,-73.60,0,Germany,1.535440,Credit Card,NaN,Electronics,Online,Not Returned,DHL,NaN,Medium,2020-01-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49691,553510,SKU_1943,T-shirt,-4,-74.57,0,France,1.306613,Credit Card,NaN,Stationery,Online,Not Returned,Royal Mail,NaN,Low,2025-09-01
49695,855821,SKU_1500,Wireless Mouse,-11,-18.88,0,Portugal,1.569082,Bank Transfer,NaN,Furniture,In-store,Not Returned,FedEx,NaN,Low,2025-09-01
49718,745888,SKU_1224,T-shirt,-2,-84.25,0,United States,1.420665,Bank Transfer,NaN,Accessories,In-store,Not Returned,Royal Mail,NaN,Medium,2025-09-02
49730,487771,SKU_1083,Wall Clock,-7,-57.31,0,United Kingdom,1.139869,paypall,NaN,Accessories,Online,Not Returned,DHL,NaN,Low,2025-09-03


In [37]:
## Checking how many rows are when the Quantity is negative and the Shipping Cost is NaN (2489 rows >> Matching total missing values on Shipping cost column)
filtered_df2 = online_sales[(online_sales['Quantity'] < 0) & (online_sales['Shipping Cost'].isna())] 

In [38]:
filtered_df2

,Invoice No,Stock Code,Description,Quantity,Unit Price,Customer ID,Country,Discount,Payment Method,Shipping Cost,Category,Sales Channel,Return Status,Shipment Provider,Warehouse Location,Order Priority,Order Date
4,359178,SKU_1386,USB Cable,-30,-68.11,0,United Kingdom,1.501433,Bank Transfer,NaN,Electronics,In-store,Not Returned,FedEx,NaN,Medium,2020-01-01
144,451279,SKU_1617,Wall Clock,-2,34.06,0,Spain,0.340000,Credit Card,NaN,Furniture,Online,Not Returned,DHL,NaN,Medium,2020-01-07
147,747972,SKU_1514,Desk Lamp,-26,-72.33,0,Netherlands,1.972087,paypall,NaN,Electronics,In-store,Not Returned,Royal Mail,NaN,Low,2020-01-07
167,318969,SKU_1115,Desk Lamp,-19,-3.61,0,Italy,1.052254,Credit Card,NaN,Apparel,In-store,Not Returned,UPS,NaN,High,2020-01-07
176,285340,SKU_1710,Blue Pen,-3,85.95,0,Netherlands,0.260000,Credit Card,NaN,Accessories,In-store,Not Returned,UPS,NaN,Low,2020-01-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49730,487771,SKU_1083,Wall Clock,-7,-57.31,0,United Kingdom,1.139869,paypall,NaN,Accessories,Online,Not Returned,DHL,NaN,Low,2025-09-03
49739,106493,SKU_1736,Desk Lamp,-40,-51.34,0,Netherlands,0.230000,Bank Transfer,NaN,Accessories,In-store,Not Returned,FedEx,NaN,Medium,2025-09-03
49749,540459,SKU_1405,White Mug,-36,12.21,0,Portugal,0.290000,paypall,NaN,Stationery,Online,Not Returned,UPS,NaN,High,2025-09-03
49760,726763,SKU_1273,White Mug,-34,10.34,0,France,0.350000,Credit Card,NaN,Furniture,Online,Not Returned,DHL,NaN,High,2025-09-04


In [39]:
## Since the negative values seem to be related to the missing shipping cost, next: 

## 1. Change values in Unit Price & Quality columns to positive
## 2. Fill nulls in Shipping Cost column

In [40]:
## 1: 

In [41]:
online_sales['Unit Price'] = online_sales['Unit Price'].abs()

online_sales['Quantity'] = online_sales['Quantity'].abs()

In [42]:
negative_prices = online_sales[online_sales['Unit Price'] < 0]

In [43]:
negative_quantities = online_sales[online_sales['Quantity'] < 0]

In [44]:
negative_prices

,Invoice No,Stock Code,Description,Quantity,Unit Price,Customer ID,Country,Discount,Payment Method,Shipping Cost,Category,Sales Channel,Return Status,Shipment Provider,Warehouse Location,Order Priority,Order Date


In [45]:
negative_quantities

,Invoice No,Stock Code,Description,Quantity,Unit Price,Customer ID,Country,Discount,Payment Method,Shipping Cost,Category,Sales Channel,Return Status,Shipment Provider,Warehouse Location,Order Priority,Order Date


In [46]:
## 2: 

In [47]:

# Calculate the mean Shipping Cost for each Country
mean_shipping_by_country = online_sales.groupby('Country')['Shipping Cost'].transform('mean')

# Fill the NaN values in Shipping Cost with the mean for the respective Country
online_sales['Shipping Cost'] = online_sales['Shipping Cost'].fillna(mean_shipping_by_country)

In [48]:
nulls_shipping_cost = online_sales[online_sales['Shipping Cost'].isnull()]

In [49]:
nulls_shipping_cost

,Invoice No,Stock Code,Description,Quantity,Unit Price,Customer ID,Country,Discount,Payment Method,Shipping Cost,Category,Sales Channel,Return Status,Shipment Provider,Warehouse Location,Order Priority,Order Date


In [50]:
online_sales

,Invoice No,Stock Code,Description,Quantity,Unit Price,Customer ID,Country,Discount,Payment Method,Shipping Cost,Category,Sales Channel,Return Status,Shipment Provider,Warehouse Location,Order Priority,Order Date
0,221958,SKU_1964,White Mug,38,1.71,37039,Australia,0.470000,Bank Transfer,10.790000,Apparel,In-store,Not Returned,UPS,London,Medium,2020-01-01
1,771155,SKU_1241,White Mug,18,41.25,19144,Spain,0.190000,paypall,9.510000,Electronics,Online,Not Returned,UPS,Rome,Medium,2020-01-01
2,231932,SKU_1501,Headphones,49,29.11,50472,Germany,0.350000,Bank Transfer,23.030000,Electronics,Online,Returned,UPS,Berlin,High,2020-01-01
3,465838,SKU_1760,Desk Lamp,14,76.68,96586,Netherlands,0.140000,paypall,11.080000,Accessories,Online,Not Returned,Royal Mail,Rome,Low,2020-01-01
4,359178,SKU_1386,USB Cable,30,68.11,0,United Kingdom,1.501433,Bank Transfer,17.387931,Electronics,In-store,Not Returned,FedEx,NaN,Medium,2020-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49777,354083,SKU_1562,Blue Pen,25,70.92,51445,Spain,0.200000,Credit Card,8.960000,Electronics,Online,Returned,UPS,Berlin,Medium,2025-09-05
49778,296698,SKU_1930,USB Cable,7,51.74,28879,United States,0.230000,Bank Transfer,23.550000,Electronics,Online,Not Returned,FedEx,Amsterdam,Low,2025-09-05
49779,177622,SKU_1766,Office Chair,43,85.25,21825,Portugal,0.200000,Bank Transfer,16.260000,Furniture,In-store,Not Returned,FedEx,London,High,2025-09-05
49780,701213,SKU_1602,Notebook,48,39.64,43199,United Kingdom,0.310000,paypall,28.560000,Apparel,Online,Not Returned,Royal Mail,London,Medium,2025-09-05


In [51]:
#Round values since some are an entire number and the others have decimals: 

online_sales['Shipping Cost'] = online_sales['Shipping Cost'].round()
online_sales['Discount'] = online_sales['Discount'].round()
online_sales['Unit Price'] = online_sales['Unit Price'].round()

In [52]:
online_sales.head()

,Invoice No,Stock Code,Description,Quantity,Unit Price,Customer ID,Country,Discount,Payment Method,Shipping Cost,Category,Sales Channel,Return Status,Shipment Provider,Warehouse Location,Order Priority,Order Date
0,221958,SKU_1964,White Mug,38,2.0,37039,Australia,0.0,Bank Transfer,11.0,Apparel,In-store,Not Returned,UPS,London,Medium,2020-01-01
1,771155,SKU_1241,White Mug,18,41.0,19144,Spain,0.0,paypall,10.0,Electronics,Online,Not Returned,UPS,Rome,Medium,2020-01-01
2,231932,SKU_1501,Headphones,49,29.0,50472,Germany,0.0,Bank Transfer,23.0,Electronics,Online,Returned,UPS,Berlin,High,2020-01-01
3,465838,SKU_1760,Desk Lamp,14,77.0,96586,Netherlands,0.0,paypall,11.0,Accessories,Online,Not Returned,Royal Mail,Rome,Low,2020-01-01
4,359178,SKU_1386,USB Cable,30,68.0,0,United Kingdom,2.0,Bank Transfer,17.0,Electronics,In-store,Not Returned,FedEx,NaN,Medium,2020-01-01


In [53]:
## Column "Payment Method": check values and correct if necessary (paypall): 

online_sales['Payment Method'].unique()

array(['Bank Transfer', 'paypall', 'Credit Card'], dtype=object)

In [54]:
online_sales['Payment Method'] = online_sales['Payment Method'].replace('paypall', 'PayPal')

In [55]:
online_sales['Payment Method'].unique()

array(['Bank Transfer', 'PayPal', 'Credit Card'], dtype=object)

In [56]:
## Column'Warehouse Location': reeplace nulls with 'Unknown': 

online_sales['Warehouse Location'] = online_sales['Warehouse Location'].fillna('Unknown')

In [57]:
online_sales['Warehouse Location'].unique()

array(['London', 'Rome', 'Berlin', 'Unknown', 'Amsterdam', 'Paris'],
      dtype=object)

In [58]:
online_sales.to_csv('Online_Sales_Final.csv')